In [1]:
from utils import *
import torch

In [2]:
# Load EEG data
# path = '/expanse/projects/nemar/child-mind-dtyoung/'
path = './data/'
winLength = 2
numChan = 24
srate = 128
feature = 'raw'
one_channel = False

role = 'train'
train_data = load_data(path, role, winLength, numChan, srate, feature, one_channel)
print(f'X_train shape: {len(train_data)}, {train_data[0][0].shape}')
print(f'Y_train shape: {len(train_data)}, {train_data[0][1].shape}')

role = 'val'
val_data = load_data(path, role, winLength, numChan, srate, feature, one_channel)
print(f'X_val shape: {len(val_data)}, {val_data[0][0].shape}')
print(f'Y_val shape: {len(val_data)}, {val_data[0][1].shape}')


X_train shape: 71381, (1, 24, 256)
Y_train shape: 71381, ()
X_val shape: 39868, (1, 24, 256)
Y_val shape: 39868, ()


In [ ]:
plot_EEG(train_data, feature, numChan)

In [4]:
USE_GPU = True

dtype = torch.float32 # we will be using float throughout this tutorial

if USE_GPU and torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

# Constant to control how frequently we print train loss
print_every = 100

print('using device:', device)

using device: cuda


In [6]:
def check_accuracy(loader, model):
    if loader.dataset.train:
        logger.log('Checking accuracy on training set')
    elif loader.dataset.val:
        logger.log('Checking accuracy on validation set')
    else:
        logger.log('Checking accuracy on test set')   
    num_correct = 0
    num_samples = 0
    model.eval()  # set model to evaluation mode
    with torch.no_grad():
        for x, y in loader:
            x = x.to(device=device, dtype=dtype)  # move to device, e.g. GPU
            y = y.to(device=device, dtype=torch.long)
            scores = model(x)
            _, preds = scores.max(1)
            num_correct += (preds == y).sum()
            num_samples += preds.size(0)
        acc = float(num_correct) / num_samples
        logger.log('Got %d / %d correct (%.2f)' % (num_correct, num_samples, 100 * acc))
        return acc

In [7]:
def train(model, optimizer, epochs=1):
    """
    Train a model on CIFAR-10 using the PyTorch Module API.
    
    Inputs:
    - model: A PyTorch Module giving the model to train.
    - optimizer: An Optimizer object we will use to train the model
    - epochs: (Optional) A Python integer giving the number of epochs to train for
    
    Returns: Nothing, but prints model accuracies during training.
    """
    model = model.to(device=device)  # move the model parameters to CPU/GPU
    for e in range(epochs):
        for t, (x, y) in enumerate(loader_train):
            model.train()  # put model to training mode
            x = x.to(device=device, dtype=dtype)  # move to device, e.g. GPU
            y = y.to(device=device, dtype=torch.long)

            scores = model(x)
            loss = F.cross_entropy(scores, y)

            # Zero out all of the gradients for the variables which the optimizer
            # will update.
            optimizer.zero_grad()

            # This is the backwards pass: compute the gradient of the loss with
            # respect to each  parameter of the model.
            loss.backward()

            # Actually update the parameters of the model using the gradients
            # computed by the backwards pass.
            optimizer.step()

            if t % print_every == 0:
                logger.writer.add_scalar("Loss/train", loss.item(), e*len(loader_train)+t)
                logger.log('Epoch %d, Iteration %d, loss = %.4f' % (e, t, loss.item()))
        train_acc = check_accuracy(loader_train, model)
        logger.writer.add_scalar("Acc/train", train_acc, e)        
        val_acc = check_accuracy(loader_val, model)
        logger.writer.add_scalar("Acc/valid", val_acc, e)        
        logger.log()
        
        # Save model every 20 epochs
        if e > 0 and e % 10 == 0:
            logger.save_model(model,f"epoch{e}")
        elif val_acc >= 0.83:
            logger.save_model(model,f"valacc83-epoch{e}")
        elif val_acc >= 0.84:
            logger.save_model(model,f"valacc84-epoch{e}")
    # save final model
    logger.save_model(model,f"epoch{e}")
    return model

In [4]:
model = create_model('vgg',feature)
print(model)
from pytorch_model_summary import summary
print(summary(model, torch.zeros((1, 1, 24, 256)), show_input=False))

Sequential(
  (features): Sequential(
    (0): Conv2d(1, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, 

In [10]:
def test_model(model, test_data, subj_csv):
    # one-segment test
    logger.log('Testing model accuracy using 1-segment metric')
    loader_test = DataLoader(test_data, batch_size=70)
    per_sample_acc = check_accuracy(loader_test, model)

    # 40-segment test
    logger.log('Testing model accuracy using 40-segment per subject metric')
    with open(subj_csv, newline='') as csvfile:
        spamreader = csv.reader(csvfile, delimiter=' ', quotechar='|')
        subjIDs = [row[0] for row in spamreader]
    unique_subjs,indices = np.unique(subjIDs,return_index=True)

    iterable_test_data = list(iter(DataLoader(test_data, batch_size=1)))
    num_correct = []
    for subj,idx in zip(unique_subjs,indices):
    #     print(f'Subj {subj} - gender {iterable_test_data[idx][1]}')
        data = iterable_test_data[idx:idx+40]
        #print(np.sum([y for _,y in data]))
        assert 40 == np.sum([y for _,y in data]) or 0 == np.sum([y for _,y in data])
        preds = []
        correct = 0
        with torch.no_grad():
            for x,y in data:
                x = x.to(device=device, dtype=dtype)  # move to device, e.g. GPU
                correct = y
                scores = model(x)
                _, pred = scores.max(1)
                preds.append(pred)
        final_pred = (torch.mean(torch.FloatTensor(preds)) > 0.5).sum()
        num_correct.append((final_pred == correct).sum())
    #print(len(num_correct))
    acc = float(np.sum(num_correct)) / len(unique_subjs)
    logger.log('Got %d / %d correct (%.2f)' % (np.sum(num_correct), len(unique_subjs), 100 * acc))
    return per_sample_acc, acc

In [11]:
def run_experiment(seed, model_name, feature, num_epoch):
    model = create_model()
    logger.set_model_save_location(f'{model_name}-{feature}')
    experiment = f'{model_name}-{feature}-seed{seed}'
    logger.set_experiment(experiment)

    np.random.seed(seed)
    torch.manual_seed(seed)

    # toggle between learning rate and batch size values 

    optimizer = torch.optim.Adamax(model.parameters(), lr=0.002, weight_decay=0.001)
    model = train(model, optimizer, epochs=num_epoch)
    
    # Testing
    logger.log('Testing on balanced test set')
    test_data_balanced = load_data(path, 'test', winLength, numChan, srate, feature,'v2')
    sample_acc1, subject_acc1 = test_model(model, test_data_balanced, path + 'test_subjIDs.csv')

    logger.log('Testing on all-male test set')
    test_data_all_male = load_data(path, 'test', winLength, numChan, srate, feature,'v3')
    sample_acc2, subject_acc2 = test_model(model, test_data_all_male, path + 'test_subjIDs_more_test.csv')
    
    return model

In [16]:
logger = Logger()
batch_size = 70 # original
loader_train = DataLoader(train_data, batch_size=batch_size, shuffle=True)
loader_val = DataLoader(val_data, batch_size=batch_size)
for s in range(2):
    model = run_experiment(s, 'vgg', 'raw', 1)

04/25 03:14:37 PM Epoch 0, Iteration 0, loss = 0.6951
04/25 03:14:38 PM Epoch 0, Iteration 100, loss = 0.5382
04/25 03:14:39 PM Epoch 0, Iteration 200, loss = 0.4879
04/25 03:14:40 PM Epoch 0, Iteration 300, loss = 0.4606
04/25 03:14:41 PM Epoch 0, Iteration 400, loss = 0.4616
04/25 03:14:42 PM Epoch 0, Iteration 500, loss = 0.4452
04/25 03:14:43 PM Epoch 0, Iteration 600, loss = 0.3575
04/25 03:14:44 PM Epoch 0, Iteration 700, loss = 0.2748
04/25 03:14:46 PM Epoch 0, Iteration 800, loss = 0.3252
04/25 03:14:47 PM Epoch 0, Iteration 900, loss = 0.3752
04/25 03:14:48 PM Epoch 0, Iteration 1000, loss = 0.4115
04/25 03:14:48 PM Checking accuracy on training set
04/25 03:14:51 PM Got 61282 / 71381 correct (85.85)
04/25 03:14:51 PM Checking accuracy on validation set
04/25 03:14:52 PM Got 31790 / 39868 correct (79.74)
04/25 03:14:52 PM 
04/25 03:14:52 PM Testing on balanced test set
X_test shape: (16006, 1, 24, 256)
Y_test shape: (16006, 1)
04/25 03:14:56 PM Testing model accuracy using 1-s

In [4]:
def test_all_seeds(model_name, epoch,isBalanced):
    if isBalanced:
        logger.log('Testing on balanced test set')
        test_data = load_data(path, 'test', winLength, numChan, srate, feature,'v2')
        subjIDs_file = 'data/test_subjIDs_fewer_subjects.csv'
    else:
        logger.log('Testing on all male test set')
        test_data = load_data(path, 'test', winLength, numChan, srate, feature,'v3')
        subjIDs_file = 'data/test_subjIDs_more_test.csv'

    sample_acc = []
    subject_acc = []
    for s in range(10):
        model = create_model()
        model.load_state_dict(torch.load(f'saved-model/{model_name}-seed{s}-epoch{epoch}'))
        model.to(device=device)
        sam_acc, sub_acc = test_model(model, test_data,subjIDs_file)
        sample_acc.append(sam_acc)
        subject_acc.append(sub_acc)
        
    sample_acc = np.multiply(sample_acc,100)
    subject_acc = np.multiply(subject_acc,100)
    return sample_acc, subject_acc

In [6]:
logger = Logger(mode='debug')
role = 'test'
test_data = load_data(path, role, winLength, numChan, srate, feature, one_channel)

sample_acc, subject_acc = test_all_seeds("saved-model/vgg-raw/model-vgg-raw", 'vgg',feature,test_data,'data/test_subjIDs_fewer_subjects.csv',10, 10,device, dtype,logger)
min_sample = np.min(sample_acc)
max_sample = np.max(sample_acc)
mean_sample = np.mean(sample_acc)
std_sample = np.std(sample_acc)

min_subj = np.min(subject_acc)
max_subj = np.max(subject_acc)
mean_subj = np.mean(subject_acc)
std_subj = np.std(subject_acc)

logger.log("Per sample")
logger.log(f"Min: {min_sample}, Max: {max_sample}, Mean: {mean_sample}, STDEV: {std_sample}")

logger.log("Per subject")
logger.log(f"Min: {min_subj}, Max: {max_subj}, Mean: {mean_subj}, STDEV: {std_subj}")


Testing model accuracy using 1-segment metric
Checking accuracy on test set
Got 13032 / 15925 correct (81.83)
Testing model accuracy using 40-segment per subject metric
Got 169 / 197 correct (85.79)
Testing model accuracy using 1-segment metric
Checking accuracy on test set
Got 13285 / 15925 correct (83.42)
Testing model accuracy using 40-segment per subject metric
Got 167 / 197 correct (84.77)
Testing model accuracy using 1-segment metric
Checking accuracy on test set
Got 13270 / 15925 correct (83.33)
Testing model accuracy using 40-segment per subject metric
Got 173 / 197 correct (87.82)
Testing model accuracy using 1-segment metric
Checking accuracy on test set
Got 13319 / 15925 correct (83.64)
Testing model accuracy using 40-segment per subject metric
Got 173 / 197 correct (87.82)
Testing model accuracy using 1-segment metric
Checking accuracy on test set
Got 13326 / 15925 correct (83.68)
Testing model accuracy using 40-segment per subject metric
Got 172 / 197 correct (87.31)
Testi

In [13]:
logger = Logger(mode='debug')
epochs = [20, 30, 40, 60, 69]

with open("vgg-raw-test-results-balanced.csv", 'w') as out:
    out.write('epoch,min_sam,max_sam,mean_sam,std_sam,min_subj,max_subj,mean_subj,std_subj\n')
    for epoch in epochs:
        sample_acc, subject_acc = test_all_seeds(model_name="vgg-raw/model-vgg-raw", epoch=epoch, isBalanced=True)

        min_sample = np.min(sample_acc)
        max_sample = np.max(sample_acc)
        mean_sample = np.mean(sample_acc)
        std_sample = np.std(sample_acc)

        min_subj = np.min(subject_acc)
        max_subj = np.max(subject_acc)
        mean_subj = np.mean(subject_acc)
        std_subj = np.std(subject_acc)

        logger.log("Per sample")
        logger.log(f"Min: {min_sample}, Max: {max_sample}, Mean: {mean_sample}, STDEV: {std_sample}")

        logger.log("Per subject")
        logger.log(f"Min: {min_subj}, Max: {max_subj}, Mean: {mean_subj}, STDEV: {std_subj}")
        out.write(f"{epoch},{min_sample},{max_sample},{mean_sample},{std_sample},{min_subj},{max_subj},{mean_subj},{std_subj}\n")
        

Testing on balanced test set
X_test shape: (16006, 1, 24, 256)
Y_test shape: (16006, 1)
Testing model accuracy using 1-segment metric
Checking accuracy on test set
Got 13209 / 16006 correct (82.53)
Testing model accuracy using 40-segment per subject metric
Got 172 / 197 correct (87.31)
Testing model accuracy using 1-segment metric
Checking accuracy on test set
Got 13278 / 16006 correct (82.96)
Testing model accuracy using 40-segment per subject metric
Got 169 / 197 correct (85.79)
Testing model accuracy using 1-segment metric
Checking accuracy on test set
Got 13357 / 16006 correct (83.45)
Testing model accuracy using 40-segment per subject metric
Got 169 / 197 correct (85.79)
Testing model accuracy using 1-segment metric
Checking accuracy on test set
Got 13389 / 16006 correct (83.65)
Testing model accuracy using 40-segment per subject metric
Got 172 / 197 correct (87.31)
Testing model accuracy using 1-segment metric
Checking accuracy on test set
Got 13316 / 16006 correct (83.19)
Testin

Got 13212 / 16006 correct (82.54)
Testing model accuracy using 40-segment per subject metric
Got 171 / 197 correct (86.80)
Testing model accuracy using 1-segment metric
Checking accuracy on test set
Got 13187 / 16006 correct (82.39)
Testing model accuracy using 40-segment per subject metric
Got 170 / 197 correct (86.29)
Testing model accuracy using 1-segment metric
Checking accuracy on test set
Got 13120 / 16006 correct (81.97)
Testing model accuracy using 40-segment per subject metric
Got 171 / 197 correct (86.80)
Testing model accuracy using 1-segment metric
Checking accuracy on test set
Got 13320 / 16006 correct (83.22)
Testing model accuracy using 40-segment per subject metric
Got 175 / 197 correct (88.83)
Per sample
Min: 81.96926152692741, Max: 83.99975009371485, Mean: 82.79270273647383, STDEV: 0.5887852538858335
Per subject
Min: 85.27918781725889, Max: 89.34010152284264, Mean: 86.95431472081216, STDEV: 1.202306526124571
Testing on balanced test set
X_test shape: (16006, 1, 24, 25